# Parameterization in Mojo

Compile time meta-programming similar to template in C++.

1. Parameter value are known at compile time. Parameter are specified using the `[]`
2. Argument value are known at runtime.

In [1]:
let x = SIMD[DType.uint8,4](5)
print(x)

[5, 5, 5, 5]


In [2]:
print(len(x))

4


In [3]:
for i in range(len(x)):
    print(x[i])

5
5
5
5


In [4]:
print(x.element_type)

uint8


In [5]:
print(x.fma(4,5))

[25, 25, 25, 25]


# Argument Control

## Python def argument is Reference Semantic

If pass the list to def function then i modified the list inside the def ,  this modification is visible outside the def also.

In [6]:
%%python
a = [1,2,3,4]
print(f"{'Before Change ':<25s}:",a)
def change(a):
    a[2]  = 30
    return a
print(f"{'Value return by def':<25s}:",change(a))
print(f"{'After Change ':<25s}:",a)


Before Change            : [1, 2, 3, 4]
Value return by def      : [1, 2, 30, 4]
After Change             : [1, 2, 30, 4]


change is persisted outside the function

## Mojo def argument is Value Semantic
def function receive the copy of the argument you can modify the value but the modification in visible to outside.

In [7]:
var mojo_simd = SIMD[DType.uint8,4](1,2,3,4)
print("Before Change            :",mojo_simd)

def def_mojo_change[ty:DType,width:Int](a:SIMD[ty,width]) -> SIMD[ty,width]:
    a[2] = 30
    return a

print("Value return by def      :",def_mojo_change(mojo_simd))
print("After Change             :",mojo_simd)

Before Change            : [1, 2, 3, 4]
Value return by def      : [1, 2, 30, 4]
After Change             : [1, 2, 3, 4]


change is not persisted outside the func

## Mojo fn argument is Immutable Reference
argument receive the original object but you cant modify the object

In [8]:
var mojo_simd = SIMD[DType.uint8,4](1,2,3,4)
print("Before Change            :",mojo_simd)

fn fn_mojo_change[ty:DType,width:Int](a:SIMD[ty,width]) -> SIMD[ty,width]:
    # uncomment this to see error
    # a[2] = 30
    print("You can read             :",a[2])
    return a

print("Value return by fn       :",fn_mojo_change(mojo_simd))
print("After Change             :",mojo_simd)

Before Change            : [1, 2, 3, 4]
You can read             : 3
Value return by fn       : [1, 2, 3, 4]
After Change             : [1, 2, 3, 4]


uncomment assign, then complier will throw error 

## Borrowed(Immutable argument)

1. fn argument are immutable , this is done by `borrowed` keyword which is default for the fn.
2. this functionally much similar to the `const &` in C++.
3. Borrowed use to control the ownership of the argument only  `read and execute` access but no `write` access.

In [9]:
var mojo_simd = SIMD[DType.uint8,4](1,2,3,4)
print("Before Change            :",mojo_simd)

# borrowed kweyword in default
fn fn_borrowed[ty:DType,width:Int](borrowed a:SIMD[ty,width]) -> SIMD[ty,width]:
    # uncomment this to see error
    # a[2] = 30
    print("You can read             :",a[2])
    return a

print("Value return by fn       :",fn_borrowed(mojo_simd))
print("After Change             :",mojo_simd)

Before Change            : [1, 2, 3, 4]
You can read             : 3
Value return by fn       : [1, 2, 3, 4]
After Change             : [1, 2, 3, 4]


## InOut (Mutable argument)

1. if the fn argument is immutable then how we are going to in place addition and other stuff.
2. This done by `inout` keyword . argument can modified INside the function and modification  is visible OUTside the function.

In [10]:
var mojo_simd = SIMD[DType.uint8,4](1,2,3,4)
print("Before Change            :",mojo_simd)

# borrowed kweyword in default
fn fn_inout[ty:DType,width:Int](inout a:SIMD[ty,width]) -> SIMD[ty,width]:
    a[2] = 30
    return a

print("Value return by fn       :",fn_inout(mojo_simd))
print("After Change             :",mojo_simd)

Before Change            : [1, 2, 3, 4]
Value return by fn       : [1, 2, 30, 4]
After Change             : [1, 2, 30, 4]


## Using MLIR Dialects inside the mojo

[Refer Index Dialects](https://mlir.llvm.org/docs/Dialects/IndexOps/)

1. Operation
2. Types
3. Attribute

In [12]:
var lhs:__mlir_type.index = __mlir_attr.`42:index`
var rhs:__mlir_type.index = __mlir_attr.`100:index`

add_result = __mlir_op.`index.add`(lhs,rhs)
print(add_result)

142


In [20]:
var int_lhs:__mlir_type.i8 = __mlir_attr.`42:i8`
var int_rhs:__mlir_type.i8 = __mlir_attr.`100:i8`

# var add_result:__mlir_type.i8 = __mlir_op.`arith.addi`[_type:__mlir_type.i8](int_lhs,int_rhs)
# this throws error **cannot be converted to LLVM IR: missing `LLVMTranslationDialectInterface` registration for dialect for op: arith.addi**

error: Expression [20]:29:81: cannot be converted to LLVM IR: missing `LLVMTranslationDialectInterface` registration for dialect for op: arith.addi
    var add_result:__mlir_type.i8 = __mlir_op.`arith.addi`[_type:__mlir_type.i8](int_lhs,int_rhs)
                                                                                ^

expression failed to parse (no further compiler diagnostics)

## Overloading

### Overloading on function arguments

In [24]:
fn fn_foo(x:Int):
    print("Calling int fn_foo")

fn fn_foo(x:Float64):
    print("Calling Float fn_foo")

fn_foo(1)
fn_foo(1.0)

Calling int fn_foo
Calling Float fn_foo


In [25]:
def def_foo(x:Int):
    print("Calling int def_foo")

def def_foo(x:Float64):
    print("Calling Float def_foo")

def_foo(1)
def_foo(1.0)

Calling int def_foo
Calling Float def_foo


### Overloading on methods

In [18]:
struct Point2d:
    var x:Int
    var y:Int

    fn __init__(inout self,x:Int):
        self.x = x
        self.y = 0
    
    fn __init__(inout self,x:Int,y:Int):
        self.x = x 
        self.y = y

    # Currently No Polymorphism in mojo 
    # https://docs.modular.com/mojo/roadmap.html#no-polymorphism
    # fn __str__(self)->Tuple[Int,Int]:
    #     return self.x,self.y

    fn print_values(self):
        return print("x:",self.x,", y:",self.y)

var only_x:Point2d = Point2d(1)
var both_xy:Point2d = Point2d(1,10)


only_x.print_values()
both_xy.print_values()

x: 1 , y: 0
x: 1 , y: 10


### Overloading on parameters

In [28]:
fn fn_param[a:Int]():
    print("Int Param")

fn fn_param[a:Float64]():
    print("Float param")

fn_param[1]()
fn_param[1.0]()

Int Param
Float param


## Implicit Conversion

In [33]:
var index_val:__mlir_type.`index` = __mlir_attr.`3:index`
var int_val:Int = index_val

print(int_val)

3


In [34]:
var float_val:Float64 = 2
print(float_val)

2.0


In [35]:
fn return_float()->Float64:
    return 4

print(return_float())

4.0
